Jesus Rodrigo Colina Nunez

Student Number: 2017156 

CA 2 Semester 2

#### Imports

In [12]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, to_timestamp, date_format, length

In [13]:
##spark.conf.set("spark.sql.debug.maxToStringFields", 100)

In [14]:
from pyspark.sql import SparkSession


In [15]:
df1 = pd.read_csv('ProjectTweets.csv')

#Load CSV

In [16]:
df1.head()

# Hhere wI show how it looks, HERE I am not using Pyspark just yet

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,1,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,2,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,3,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,4,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,5,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [17]:
df = spark.read.format('csv').option('header', 'false').load('/user1/ProjectTweets.csv')

#I load the data set into Pyspak in order to read it

In [18]:
from pyspark.sql.functions import countDistinct
from pyspark.ml.stat import Correlation
from pyspark.ml.feature import VectorAssembler

#More imports

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [19]:

df.show()

#Here I show the data set and we see that it does not have columns


+---+----------+--------------------+--------+---------------+--------------------+
|_c0|       _c1|                 _c2|     _c3|            _c4|                 _c5|
+---+----------+--------------------+--------+---------------+--------------------+
|  0|1467810369|Mon Apr 06 22:19:...|NO_QUERY|_TheSpecialOne_|@switchfoot http:...|
|  1|1467810672|Mon Apr 06 22:19:...|NO_QUERY|  scotthamilton|is upset that he ...|
|  2|1467810917|Mon Apr 06 22:19:...|NO_QUERY|       mattycus|@Kenichan I dived...|
|  3|1467811184|Mon Apr 06 22:19:...|NO_QUERY|        ElleCTF|my whole body fee...|
|  4|1467811193|Mon Apr 06 22:19:...|NO_QUERY|         Karoli|@nationwideclass ...|
|  5|1467811372|Mon Apr 06 22:20:...|NO_QUERY|       joy_wolf|@Kwesidei not the...|
|  6|1467811592|Mon Apr 06 22:20:...|NO_QUERY|        mybirch|         Need a hug |
|  7|1467811594|Mon Apr 06 22:20:...|NO_QUERY|           coZZ|@LOLTrish hey  lo...|
|  8|1467811795|Mon Apr 06 22:20:...|NO_QUERY|2Hood4Hollywood|@Tatiana_K nop

In [20]:
df.printSchema()

#Inspect the DataFrame's Structure:  It reveals the blueprint of your data



root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)



In [21]:
columns = ["index", "id", "timestamp", "query", "username", "tweet"]
for i in range(len(df.columns)):
    df = df.withColumnRenamed("_c" + str(i), columns[i])
    
#I am going to change the name of the columns

In [22]:
df.printSchema()
# This schema provides a valuable overview for understanding the structure and content of my dataset 
# before I start analyzing or manipulating it.

root
 |-- index: string (nullable = true)
 |-- id: string (nullable = true)
 |-- timestamp: string (nullable = true)
 |-- query: string (nullable = true)
 |-- username: string (nullable = true)
 |-- tweet: string (nullable = true)



In [23]:
df.show(10, truncate=False)

#Here I show the 10 first rows and we noticed that the names has changed

+-----+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|index|id        |timestamp                   |query   |username       |tweet                                                                                                              |
+-----+----------+----------------------------+--------+---------------+-------------------------------------------------------------------------------------------------------------------+
|0    |1467810369|Mon Apr 06 22:19:45 PDT 2009|NO_QUERY|_TheSpecialOne_|@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D|
|1    |1467810672|Mon Apr 06 22:19:49 PDT 2009|NO_QUERY|scotthamilton  |is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!    |
|2    |1467810917|Mon Apr 06 22:19:53 PDT 2009|NO_QUERY

In [25]:



# Get summary statistics count, mean
df.describe().show()

+-------+------------------+--------------------+--------------------+--------+--------------------+--------------------+
|summary|             index|                  id|           timestamp|   query|            username|               tweet|
+-------+------------------+--------------------+--------------------+--------+--------------------+--------------------+
|  count|           1600000|             1600000|             1600000| 1600000|             1600000|             1600000|
|   mean|          799999.5|1.9988175522956276E9|                null|    null| 4.325887521835714E9|                null|
| stddev|461880.35968924535|1.9357607362268043E8|                null|    null|5.162733218454890...|                null|
|    min|                 0|          1467810369|Fri Apr 17 20:30:...|NO_QUERY|        000catnap000|                 ...|
|    max|            999999|          2329205794|Wed May 27 07:27:...|NO_QUERY|          zzzzeus111|ï¿½ï¿½ï¿½ï¿½ï¿½ß§...|
+-------+---------------

In [19]:
# Add a column for tweet length
df = df.withColumn("tweet_length", length(col("tweet")))

# Analyze tweet lengths (min, max, mean)
df.describe("tweet_length").show()

+-------+-----------------+
|summary|     tweet_length|
+-------+-----------------+
|  count|          1600000|
|   mean|     74.090110625|
| stddev|36.44113790653486|
|    min|                6|
|    max|              374|
+-------+-----------------+



In [21]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, to_timestamp, date_format, length

# Option 1: Set legacy parser globally (recommended for quick fix)
spark = SparkSession.builder\
    .appName("TwitterEDA")\
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY")\
    .getOrCreate()


# Convert timestamp to a datetime format
df = df.withColumn("timestamp_dt", to_timestamp(col("timestamp"), "EEE MMM dd HH:mm:ss zzz yyyy")) Explicit format string


# Extract day of week, hour, and date
df = df.withColumn("day_of_week", date_format("timestamp_dt", "EEEE"))\
       .withColumn("hour", date_format("timestamp_dt", "HH"))\
       .withColumn("date", date_format("timestamp_dt", "yyyy-MM-dd"))

# Count tweets by day of week and hour
df.groupBy("day_of_week").agg(count("*").alias("tweet_count")).orderBy(col("tweet_count").desc()).show()
df.groupBy("hour").agg(count("*").alias("tweet_count")).orderBy(col("hour")).show()

# This tells Spark exactly how to interpret your timestamp string, ensuring consistent behavior 


24/05/23 01:25:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+-----------+-----------+
|day_of_week|tweet_count|
+-----------+-----------+
|     Sunday|     382894|
|     Monday|     302830|
|   Saturday|     300944|
|    Tuesday|     226527|
|     Friday|     170511|
|  Wednesday|     120866|
|   Thursday|      95428|
+-----------+-----------+



+----+-----------+
|hour|tweet_count|
+----+-----------+
|  00|      55720|
|  01|      51843|
|  02|      53485|
|  03|      57722|
|  04|      57059|
|  05|      68964|
|  06|      78328|
|  07|      84750|
|  08|      80865|
|  09|      75268|
|  10|      73991|
|  11|      74253|
|  12|      76995|
|  13|      78623|
|  14|      80852|
|  15|      83654|
|  16|      76287|
|  17|      67278|
|  18|      60689|
|  19|      61009|
+----+-----------+
only showing top 20 rows



### MontoDB Connection

In [26]:
#pip install pymongo==3.12.3  # Example: Install PyMongo version 3.12.3

# I had to install this version because The newest verion gave me lots of errors

In [27]:
from pymongo import MongoClient

# My MongoDB connection details
client = MongoClient("mongodb://localhost:27017/") 
db = client["ProjectTweets"]  # My data base name

In [28]:
import pandas as pd

df5 = pd.read_csv('ProjectTweets.csv')  # Replace with your file path

In [29]:
data = df5.to_dict(orient='records')

In [30]:
collection = db["my_collection"] 

In [31]:
import pandas as pd
from pymongo import MongoClient

# Connect to MongoDB
client = MongoClient("mongodb://localhost:27017/")
db = client["my_database"]
collection = db["my_collection"]

# Read CSV file
df = pd.read_csv('ProjectTweets.csv')

# Convert to dictionaries
data = df.to_dict(orient='records')

# Insert into MongoDB
collection.insert_many(data)

print("Data uploaded successfully!")


Data uploaded successfully!


In [32]:
# Retrieve all documents
all_documents = collection.find()

# Display the first few documents (e.g., the first 5)
for doc in all_documents.limit(5):
    print(doc)
    
#Now we can see that mongoDB is working fine and is able to read the file.

{'_id': ObjectId('664eada25c8a34c9afff91d9'), '0': 1, '1467810369': 1467810672, 'Mon Apr 06 22:19:45 PDT 2009': 'Mon Apr 06 22:19:49 PDT 2009', 'NO_QUERY': 'NO_QUERY', '_TheSpecialOne_': 'scotthamilton', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D": "is upset that he can't update his Facebook by texting it... and might cry as a result  School today also. Blah!"}
{'_id': ObjectId('664eada25c8a34c9afff91da'), '0': 2, '1467810369': 1467810917, 'Mon Apr 06 22:19:45 PDT 2009': 'Mon Apr 06 22:19:53 PDT 2009', 'NO_QUERY': 'NO_QUERY', '_TheSpecialOne_': 'mattycus', "@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D": '@Kenichan I dived many times for the ball. Managed to save 50%  The rest go out of bounds'}
{'_id': ObjectId('664eada25c8a34c9afff91db'), '0': 3, '1467810369': 1467811184, 'Mon Apr 06 22:19:45 PDT 2009': 'Mon Apr 06 22:19:57 PDT 2009', 'NO_QUE